In [1]:
from typing import Literal
from src.factor.optimizer.util import SolverInput , SolveCond , SolveVars
from src.factor.optimizer.util.solver_input import LinearConstraint , BoundConstraint , CovConstraint , TurnConstraint , ShortConstraint
from src.factor.basic.var import SYMBOL_STOCK_LB , SYMBOL_STOCK_UB

input = SolverInput.rand()




In [2]:
from src.factor.optimizer.solver import MosekSolver , CvxpySolver

In [3]:
msolver = MosekSolver(input)
msolver.solve()

(array([0.2318499 , 0.54128885, 0.22686125]), True, 'optimal')

In [6]:
csolver = CvxpySolver(input , cvxpy_solver='clarabel')
csolver.solve()

(array([0.2318499 , 0.54128885, 0.22686124]), True, 'optimal')